In [1]:
from __future__ import print_function

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import numpy.random as rnd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, Image

# Interactive Python Plotting with Jupyter
## Jupyter Lab is a **much** more flexible version of notebook
The lab allows for multiple tabs, easier use of widgets, separate views for output cells, etc

## Interactive Plotting
In Jupyter labs/notebook, the interactive plotting is done by linking widgets to the plots
FYI: The backend up Jupyter is **javascript** - so there is a rapidly building library of javascript based widgets and plotting packages
### ipywidgets:
- Base documentation: <https://ipywidgets.readthedocs.io>
- Widget list: <https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html>

### plot.ly:
Why Plot.ly?
- I prefer plot.ly over other interactive libraries (like Bokeh) mostly for it's ease of use, and the visual style of the plots. Plot.ly was designed for large data in mind, and has many different kinds of plots, **including** Seaborn style plots!
- They also have their own web framework (Dash - mentioned below) which makes deploying apps online much easier than bokeh (which requires other things and more fiddling)
- Main site: <https://plot.ly/python/> (Note: Plot.ly has libraries for R, Python, (some in Julia), Pandas, Matlab, and Javascript)
- 3.0 site: https://github.com/plotly/plotly.py

# Let's get started with a simple example using matplotlib and ipywidgets
Generate some data with spread and some kind of 'trend' in logspace

In [2]:
#Generate some 'random' data with a trend
logx = rnd.uniform(0, 3, 256)
logy = 1.2*logx + 0.4 + rnd.normal(0, 1, 256)
x = 10**logx; y = 10**logy
xline = 10**np.linspace(0, 3, 3)

## How to use ipywidgets - simple form
The most straightforward way to use ipywidgets is to use the @interact wrapper
You define some function that will have the interactive behavior (such as making a plot)
ipywidgets is **smart**: it will generate widgets based on what you pass it! Ints become IntSliders, floats become FloatSliders, True/False are checkboxes, etc

Usage:
```python
@interact(a = ?, b = ?, c = ?)
def function(a, b, c):
    put stuff in here
```

In [3]:
@interact(m = (-2, 2, 0.1), b = (0.01, 1, 0.01), t = True)
def update_plot(m=1., b=1., t = True):
    yline = 10**(m*np.log10(xline) + b)
    plt.figure(figsize=(8,8))
    plt.loglog(x,y, 'k.')
    plt.loglog(xline, yline, 'b-', linewidth=2)
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='m', max=2.0, min=-2.0), FloatSlider(value=1.0, descr…

### Can also use the interactive() function
This form provides a bit more flexibility. You can directly declare specific types of widgets
You can customize to override the defaults which are based on the operations, as shown below
*Below*:
I've changed the @interact with calling the interactive function which works as:
interactive(func, widget1, widget2, .... )
I've overriden the float slider used for the slope above to force it to be a FloatSlider

In [4]:
def update_plot(m=1., b=1., t = True):
    yline = 10**(m*np.log10(xline) + b)
    plt.figure(figsize=(8,8))
    plt.loglog(x,y, 'k.')
    plt.loglog(xline, yline, 'b-', linewidth=2)
    plt.show()

interactive(update_plot, m = widgets.FloatSlider(min=-2,max=2,step=0.1), b = (0.01, 1, 0.01), t = True)

interactive(children=(FloatSlider(value=0.0, description='m', max=2.0, min=-2.0), FloatSlider(value=1.0, descr…

### The slider's backend is entirely in javascript
The backend of jupyter is javascript and so are the widgets! Therefore, the values are synched. Change the slider, then check the value. You can use this to sync
multiple widgets together if you wish (not shown here)

In [5]:
test = widgets.IntSlider(min=1, max=10,step=1)
test

IntSlider(value=1, max=10, min=1)

In [6]:
test.value

1

## Now on to plot.ly
Matplotlib is good for some things, but it is still fairly restrictive and becomes slow for many data points
Plot.ly 3.0 introduced a large amount of infrastructure for offline jupyter notebooks. You *can* use the online plot.ly functions - this will host your data on their servers and their plots. From their servers, if you have an account (can do free, academic accounts for students cost 99 USD), you can directly embed them into websites and explore the data on their site.

Some good videos discussing the new plot.ly 3.0:  
<https://www.youtube.com/watch?v=1ndo6C1KWjI>  
<https://www.youtube.com/watch?v=nNrsoSNx8E4>

Note: The Enthought site (and youtube in general) have a lot of good videos on interactive python plotting and web-based python apps

All of plot.ly images in notebooks start with the go.FigureWidget(). Plot.ly renders things as you add it - meaning you can customize and tweak your plot very easily and it's **fully** reactive!

In [7]:
f = go.FigureWidget() #Create the figure
data = f.add_scattergl(x=x, y=y, mode='markers', marker={'size':6, 'color':'blue'}) # Add in a scatter plot with the data. Notice how everything operates like a dictionary
line = f.add_scatter() #Add empty line (for later!)
f #Display the plot

FigureWidget({
    'data': [{'marker': {'color': 'blue', 'size': 6},
              'mode': 'markers',
        …

### tab complete + full documentation now!!
Below, start with f.add_ and tab. That shows all the kinds of plots that allow you to directly add them with the plot.ly 3.0 formulism
Inside of add_trace() try doing shift+tab - full documentation strings! You can use this to figure out what you need

In [ ]:
f.add_trace()

Plot.ly is reactive - you can modify the plot here and it changes the plot that was displayed above (or in the output view if you made one). No need to re-render the plot!

In [8]:
f.layout.xaxis.type = 'log'
f.layout.yaxis.type = 'log'

In [9]:
f.layout.xaxis.exponentformat = 'power'
f.layout.yaxis.exponentformat = 'power'

Now we can add in the interactive. Remember that empty line trace added above? That comes in handy! You don't have to recreate the trace every time, you recreate the plot.  
Just tell plot.ly to change the data within that trace and the properties of the lines.   
The use of scattergl over scatter renders things with WebGL making it MUCH faster if you go to hundreds of thousands or millions of points!

In [10]:
@interact(m = (-2, 2, 0.1), b = (0.01, 1, 0.01))
def update_plot(m=1., b=1.):
    yline = 10**(m*np.log10(xline) + b)
    line.x = xline
    line.y = yline
    line.mode = 'lines'
    line.line.color = 'magenta'
    line.line.width = 3

interactive(children=(FloatSlider(value=1.0, description='m', max=2.0, min=-2.0), FloatSlider(value=1.0, descr…

Another strong example of the reactive nature of plot.ly is you can color the points after you plot them. You don't have to recreate the trace or anything. Just access the marker property of the trace and set the color, colorscale, etc. Again, try doing tab complete on things. You can also pass a bad colormap and it will tell you the allowed choices - rather than just a vague error

In [11]:
z = x**(-0.1)
z += rnd.normal(0, 0.01, len(z))

In [12]:
data.marker.color = z
data.marker.colorscale = 'Viridis'
data.marker.reversescale = True

Color is nice...but what about other information?? Add in hover text!

In [13]:
data.hoverinfo = 'text'
data.hovertext = z.astype('str')
f.layout.hovermode = 'closest'

In [14]:
f.layout.width = 600
f.layout.height = 600
f.layout.xaxis.title = "X"
f.layout.yaxis.title = "Y"

# Building a Jupyter dashboard with **real** data!
I will use a subset of the Hypatia catalog <http://www.hypatiacatalog.com>, which has the metal abundances and properties of thousands of stars along with information on exo-planets, etc.
Use pandas to read in the csv file generated from their website

In [15]:
df = pd.read_csv('hypatia-11092018.csv')
print(list(df))

['f_hip', 'f_hd', 'f_2mass', 'Fe', 'C', 'O', 'Mg', 'Si', 'Ca', 'Ti', 'Li', 'N', 'F', 'Na', 'Al', 'P', 'K', 'Co', 'Ni', 'Cu', 'Zn', 'Be', 'S', 'f_ra', 'f_dec', 'f_x', 'f_y', 'f_z', 'f_dist', 'f_disk', 'f_spec', 'f_vmag', 'f_bv', 'f_u', 'f_v', 'f_w', 'f_teff', 'f_logg', 'f_mass', 'f_name', 'f_p', 'f_m_p', 'f_e', 'f_a', 'Unnamed: 44']


### Let's do a quick plot of the data so we know what we're working with.   
Notice, we can make the hover text the spectral type of the star!  
Then, we can colorcode the points by their distance. This gives us a good idea what we might want a dashboard to look like

In [16]:
f2 = go.FigureWidget()
data = f2.add_scattergl(x = df['C'], y = df['O'], mode = 'markers', hoverinfo = 'text', hovertext = df['f_spec'])
f2

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': array(['K1III ', 'K0 ', 'K0 ', ..…

In [17]:
f2.layout.xaxis.title = '[C/Fe]'
f2.layout.yaxis.title = '[O/Fe]'
f2.layout.width = 600
f2.layout.height = 600

In [18]:
data.marker.color = df['f_dist']
data.marker.colorscale = 'Blackbody'

Create a new pandas column - only care about the general spectral type  
Then just make a quick dict (for the dashboard)  
**Note**: The dashboard shown below was made over ~45 minutes of fiddling with things to make it look as I want, etc. In general, once you're used to doing this, it takes about 10-15 minutes to do this.  

**Why do this?** The typical thing to do is to make these plots and just replot things to find and explore the trends. Here, you can change what you're plotting, how you're coloring it, filting out the data, all at once to explore the data and make the "random" plotting more viable.

In [19]:
df['GenType'] = df['f_spec'].astype(str).str[0]

In [20]:
typeDict = {g.upper():i for (i,g) in enumerate(df['GenType'].unique())}
print(typeDict)

{'K': 4, 'M': 1, 'F': 2, 'G': 3}


## The Strength of ipywidgets  
Here we make all the widgets we think we might want. Maybe there's a trend with spectral type? Or with specific elements? Maybe there's a distance effect?  
Below we make 5 widgets and a "label". The widget information is entirely accessed through the "value" tag! First we make all the widgets - no need to display them yet.  
The dashboard won't use interact or interactive - instead we'll make the function to plot, and then force the widgets to "listen" to changes and **react**  

In [21]:
eles = ['Fe', 'C', 'O', 'Mg', 'Si', 'Ca', 'Ti', 'Li', 'N', 'F', 'Na', 'Al', 'P', 'K', 'Co', 'Ni', 'Cu', 'Zn', 'Be', 'S']
typeDrop = widgets.SelectMultiple(options=list(typeDict.keys()), value=list(typeDict.keys()), description="Spectral Type")
ele1Drop = widgets.Dropdown(options=eles,value='C', description="X")
ele2Drop = widgets.Dropdown(options=eles, value='O', description="Y")
distRange = widgets.FloatRangeSlider(min=df['f_dist'].min(), max=df['f_dist'].max(), description="Distance Range")
colorPick = widgets.Dropdown(options=['f_dist', 'f_vmag', 'f_bv', 'f_teff', 'f_logg', 'f_mass'], value='f_dist')
label = widgets.Label(value="Number of Objects: ")

Make the plot.ly figure and add the kinds of data we want. I added in a 2d histogram contour since after playing with it I thought "Hey, sometimes there's thousands of points here, let's make the plot more informative and histogram it up!"

In [22]:
f3 = go.FigureWidget()
f3.layout.width = 500
f3.layout.height = 500

In [23]:
hscat = f3.add_scattergl()
hcont = f3.add_histogram2dcontour()

In [24]:
hscat.mode = 'markers'

This is the meat of the interactive plotting. It's a function that tell's the widgets HOW to respond to a change. I made this one function, but there's no restrictions. Different widgets can respond with different functions. However, at least for ipywidgets - only 1 response function per widget. (You cannot make a function react with 2+ functions on a response) 

### I have my function, how what? 
First: The "change" function isn't used here. I haven't really ever seen it used - but it basically just stores information on the past values and current value.  
Once you have your function(s), you make them respond with:
```python
WIDGET.observe(func, names = variable)
```
the variable here is "value", this is probably what you will use. I never saw any examples of the other possibilities used.  
And that's it! Well, now you have to make them visible....

In [25]:
def update_plot(change):
    filtLow = df['f_dist'] >= distRange.value[0]
    filtHigh= df['f_dist'] <= distRange.value[1]
    filt = filtLow & filtHigh
    typeFilt = df['GenType'] == typeDrop.value[0]
    if len(typeDrop.value) > 1:
        for e in typeDrop.value[1:]:
            typeFilt = typeFilt | (df['GenType'] == e)
    totalFilt = filt & typeFilt
    tf = df[totalFilt]
    hscat.x = tf[ele1Drop.value]
    hscat.y = tf[ele2Drop.value]
    if len(tf) > 100:
        hcont.x = tf[ele1Drop.value]
        hcont.y = tf[ele2Drop.value]
        hscat.marker.opacity = 0.45
        hscat.marker.size = 4
    else:
        hcont.x = None
        hcont.y = None
    hcont.colorscale = 'Greys'
    hcont.reversescale = True
    hscat.marker.color = tf[colorPick.value]
    hscat.marker.colorscale = 'Viridis'
    hscat.hoverinfo = 'text'
    hscat.hovertext = tf['f_spec']
    label.value = "Number of Objects: %d"%(len(tf))
    f3.layout.xaxis.title = "[%s/H]"%ele1Drop.value
    f3.layout.yaxis.title = "[%s/H]"%ele2Drop.value
    
typeDrop.observe(update_plot, names='value')
ele1Drop.observe(update_plot, names='value')
ele2Drop.observe(update_plot, names='value')
distRange.observe(update_plot, names='value')
colorPick.observe(update_plot, names='value')

A cool thing ipywidgets have is VBoxes and HBoxes. This means you can create dashboards (you can do this entirely within plot.ly using their online paid service btw)

In [26]:
widgets.VBox([widgets.HBox([typeDrop, distRange]), widgets.HBox([ele1Drop, ele2Drop, colorPick]), label, f3])

In [27]:
f3.layout.width= 750
f3.layout.height = 550

# Onto Dash
Dash is plot.ly's framework for making web applications. It's fairly intuitive to use, but it can take time basically to make it look how you want.
No time to go into it! However:  
<https://dash.plot.ly/>  
<https://www.youtube.com/watch?v=sea2K4AuPOk>  

I've used this for a couple of web applications:  
<http://crgen.brandt-gaches.space>  
<http://protostarcrs.brandt-gaches.space>  

## Dash + heroku  
I deploy my apps on Heroku. The backend up Dash is called flask, and Heroku works very well with that  
<http://www.heroku.com>  
You can have a **free** account, although data is limited and the "gyros" you use are limited (you have a certain amount of hours a month free and they go inactive after 30 minutes on no use). Student accounts through github get a free "Hobby" gyro.

## Ask me more if you want to know more about this! 

# Aside: Jupyter cell magics!

In [28]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python2  %%

For instance, can do latex entirely in the cell!

In [30]:
%%latex
$x^2$
\begin{equation}
\frac{\partial \rho}{\partial t} + \nabla \cdot \left ( \rho \vec{v} \right ) = 0
\end{equation}

<IPython.core.display.Latex object>

# Aside : Selection of Points
### Shamelessly adapted from <https://github.com/jonmmease/plotly_ipywidget_notebooks>

In [54]:
from plotly.callbacks import Points
f5 = go.FigureWidget()
data1 = f5.add_scattergl(x = x, y = y, mode = 'markers')
f6 = go.FigureWidget()
data2 = f6.add_scattergl(x=x, y=z, mode='markers')

selected = np.zeros(x.size)
data1.marker.color = selected
data1.marker.colorscale = [[0, 'lightgray'], [0.5, 'lightgray'], [0.5, 'red'], [1, 'red']]
data2.marker = data1.marker

trace, points = data1, Points()

In [59]:
# Configure brush on both plots to update both plots
def brush(trace, points, state):
    inds = np.array(points.point_inds)
    if inds.size:
        selected = data1.marker.color.copy()
        selected[inds] = 1
        data1.marker.color = selected
        data2.marker.color = selected    
    
data1.on_selection(brush)
data2.on_selection(brush)

# Reset brush
def reset_brush(btn):
    selected = np.zeros(x.size)
    data1.marker.color = selected
    data2.marker.color = selected
    data1.marker.opacity = data2.marker.opacity

In [60]:
button = widgets.Button(description="clear")
button.on_click(reset_brush)
f5.layout.dragmode = 'lasso'
f6.layout.dragmode = 'lasso'

In [57]:
f5.layout.xaxis.type = 'log'
f5.layout.yaxis.type = 'log'

In [61]:
widgets.VBox([widgets.HBox([f5, f6]), button])

    'data': [{'marker': {'color': array([0., 0., 0., ..., 0., 0.,…